In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

import os
import sys
import time
import math
import numpy as np
from PIL import Image

import torch.nn as nn
import torch.nn.init as init
from torch.autograd import Variable

from scipy.ndimage import rotate

## Outstanding Questions

1. What transformations should we apply to CIFAR-10 clean data?

## Obtain CIFAR-10 Clean Data

In [7]:
# PARAMS
batch_size = 256

In [8]:
# TODO: should we apply the below transformations?
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),

])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),

])

In [10]:
print('==> Preparing data..')
trainset = torchvision.datasets.CIFAR10(root='../00_data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=16)

testset = torchvision.datasets.CIFAR10(root='../00_data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


## Create Patch - test


In [12]:
h = 'left'
v = 'center'
ratio = 3

PATCH_PATH = '../00_data/patches/toaster_patch.png'
src_img = '../00_data/test_images/banana.jpeg'
save_path = '../00_data/test_images/banana-patched.jpeg'

mode = {
    'h': h,
    'v': v
}

print(f"Path will be placed {mode['h']}-{mode['v']}")

# map from mode value to a function that returns the correct value position
mode_to_value = {
    'left': lambda width, path: 0,
    'center_h': lambda width, path: width // 2 - path.width // 2,
    'center_v': lambda height, path: height // 2 - path.height // 2,
    'right': lambda width, path: width - path.width,
    'up': lambda height, path: 0,
    'down': lambda height, path: height - path.height,
}

def get_position(mode, image, path):
    x = mode_to_value[mode['h'] if mode['h'] != 'center' else 'center_h'](image.width, path)
    y = mode_to_value[mode['v'] if mode['v'] != 'center' else 'center_v'](image.height, path)
    return x, y

patch_image = Image.open(PATCH_PATH)
src_image = Image.open(src_img)

patch_image = patch_image.resize((src_image.width // ratio, src_image.height // ratio), Image.Resampling.LANCZOS)
position = get_position(mode, src_image, patch_image)

src_image.paste(patch_image, position, mask = patch_image)
src_image.save(save_path)

print(f"Image saved at: {save_path}")

Path will be placed left-center
Image saved at: ../00_data/test_images/banana-patched.jpeg
